In [37]:
BASE_DIR = '/Users/efraflores/Desktop'

In [38]:
from IPython.display import display

from pathlib import Path
from pandas import DataFrame, read_csv

BASE_DIR = Path(BASE_DIR)
list(BASE_DIR.glob('*.csv'))

[PosixPath('/Users/efraflores/Desktop/2020.csv'),
 PosixPath('/Users/efraflores/Desktop/2021.csv'),
 PosixPath('/Users/efraflores/Desktop/2022.csv'),
 PosixPath('/Users/efraflores/Desktop/2015.csv')]

In [39]:
df = DataFrame()

for chunk_file in BASE_DIR.glob('*.csv'):
    df = df.append(read_csv(chunk_file))

print(df.shape)
df.head()

(4286391, 9)


,Unnamed: 0,Shopper Driver ID,Status,Delivered at Local Time Dynamic,City Code,Orders Count,Avg. Customer Rating,Sum of Driver Setup Distance (Km.),Sum of Delivery Distance (Km.)
0,1,502493.0,ACTIVE,2020-09-15,MEX,19,5.00,0.0,138.678033
1,2,2078542.0,BLOCKED,2020-10-26,MTY,18,5.00,0.0,80.204425
2,3,3022345.0,ACTIVE,2020-08-15,MTY,17,4.89,0.0,92.545181
3,4,2157922.0,ACTIVE,2020-10-11,MEX,17,5.00,0.0,80.688900
4,5,502493.0,ACTIVE,2020-11-26,MEX,17,5.00,0.0,85.549692


In [40]:
df['Status'].value_counts(1)

ACTIVE      0.756209
BLOCKED     0.242319
INACTIVE    0.001472
Name: Status, dtype: float64

In [41]:
df = df[(df['Status']=='ACTIVE')&(df['Shopper Driver ID'].notnull())].copy()
df.shape

(3241088, 9)

In [42]:
df['sum_rating'] = df['Orders Count']*df['Avg. Customer Rating']

aux = []
for row_date in df['Delivered at Local Time Dynamic']:
    if row_date >= '2022-04-01': aux.append('03_Apr22')
    elif row_date >= '2022-01-01': aux.append('02_2022')
    else: aux.append('01_ALL')

df['date_cut'] = aux

In [43]:
aux_df = df.copy()

In [44]:
df = df.pivot_table(index='Shopper Driver ID', columns='date_cut', aggfunc={'Orders Count':sum, 'sum_rating':sum, 'Sum of Delivery Distance (Km.)':sum})
df.columns = [f'{y} - {x}' for x,y in df.columns]
df = df[sorted(df.columns)].reset_index().copy()
df.head()

,Shopper Driver ID,01_ALL - Orders Count,01_ALL - Sum of Delivery Distance (Km.),01_ALL - sum_rating,02_2022 - Orders Count,02_2022 - Sum of Delivery Distance (Km.),02_2022 - sum_rating,03_Apr22 - Orders Count,03_Apr22 - Sum of Delivery Distance (Km.),03_Apr22 - sum_rating
0,6.0,4.0,0.000000,19.00,NaN,NaN,NaN,NaN,NaN,NaN
1,44.0,1.0,0.000000,5.00,NaN,NaN,NaN,NaN,NaN,NaN
2,993.0,3504.0,25373.455889,16130.15,55.0,581.708791,213.0,13.0,101.86116,30.0
3,2462.0,2175.0,7767.559627,9819.24,NaN,NaN,NaN,NaN,NaN,NaN
4,3913.0,1.0,0.000000,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df.to_csv(BASE_DIR.joinpath('top_shoppers.csv'), index=False)